In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from enum import Enum
import datetime
import gym
from gym import spaces
#from dqn import DQN
#from utils import tradeclass
#from utils import BCTrade

from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray
from azureml.core.environment import Environment

In [9]:
from files import BCTrade_utils

df = BCTrade_utils.getData()
del df['ask']
del df['bid']
df.to_csv(path_or_buf="data.csv")

In [5]:
from azureml.core import Workspace, Dataset

subscription_id = '736ac584-c533-43a9-8a83-d19bf290ffb8'
resource_group = 'ML-RG'
workspace_name = 'ML-Workspace'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='btc')
df = dataset.to_pandas_dataframe()
del df['date']
df['ask'] = df['price']*1.000
df['bid'] = df['price']*1.000
df = df.reindex(columns=['price','ask','bid','volume'])
df 

,price,ask,bid,volume
2015-03-17 18:27:49,290.00,290.00,290.00,3.045430
2015-03-18 13:31:32,290.00,290.00,290.00,0.042517
2015-03-19 10:12:58,290.00,290.00,290.00,2.521450
2015-03-19 10:20:03,290.00,290.00,290.00,0.625478
2015-03-19 10:20:23,290.00,290.00,290.00,4.521450
...,...,...,...,...
2018-07-27 21:48:25,7049.04,7049.04,7049.04,0.001698
2018-07-27 21:49:15,7049.04,7049.04,7049.04,0.200000
2018-07-27 21:49:15,7049.05,7049.05,7049.05,0.001698
2018-07-27 21:49:16,7049.05,7049.05,7049.05,0.198302


In [6]:
from files import BCTrade_utils

BCTrade_utils.registerBCTrade()

In [7]:
import ray
from ray import tune

#ray.init()
tune.run(
    "PPO",
    stop={"training_iteration": 10,"episode_reward_mean": 10},
    checkpoint_at_end=True,
    config={
        #"env": "CartPole-v0",
        "env": "BCTrade",
        "num_gpus": 0,
        "num_workers": 1,
        #"keep_checkpoints_num":1,
       
        #"lr": tune.grid_search([0.01]),
        #"lr": tune.grid_search([0.01, 0.001, 0.0001]),
    },
)

Trial name,status,loc
PPO_BCTrade_00cca_00000,PENDING,


(pid=10215) 2021-12-29 10:56:45,935	INFO trainer.py:753 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=10215) 2021-12-29 10:56:45,935	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=10215) 2021-12-29 10:56:45,935	INFO trainer.py:772 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=10216) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=10216)   "Box bound precision lowered by casting to {}".format(self.dtype)
(pid=10216) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops)

Trial name,status,loc
PPO_BCTrade_00cca_00000,RUNNING,10.0.0.4:10215


Trial name,status,loc
PPO_BCTrade_00cca_00000,RUNNING,10.0.0.4:10215
